In [39]:
import pandas as pd
import pickle
from llama_index.core import VectorStoreIndex, Settings
from llama_index.core.schema import TextNode, NodeRelationship, RelatedNodeInfo
from GCP.lib.LlamaIndex_custom.VertexIEmbeddings import VertexIEmbeddings


In [40]:
df = pd.read_csv('./data/Metadata/Bloques_y_tablas_v2-Retrieval-Augmented Generation for Knowledge-Intensive NLP Tasks.csv',index_col=0)
df

,block_type,text,file_type,languages,page_number,filename,description,authors,research_lab,publication_year,title,section_title
4,Author Names,"Patrick Lewis'?, Ethan Perez*,",application/pdf,['eng'],1,Retrieval-Augmented Generation for Knowledge-I...,"Lists the names of authors, potentially with a...","Patrick Lewis,Ethan Perez,Aleksandra Piktus,Fa...","Facebook AI Research,University College London...",2020,Retrieval-Augmented Generation for Knowledge-I...,Retrieval-Augmented Generation for Knowledge-I...
5,Author Names,"Aleksandra Piktus†, Fabio Petroni†, Vladimir K...",application/pdf,['eng'],1,Retrieval-Augmented Generation for Knowledge-I...,"Lists the names of authors, potentially affili...","Patrick Lewis,Ethan Perez,Aleksandra Piktus,Fa...","Facebook AI Research,University College London...",2020,Retrieval-Augmented Generation for Knowledge-I...,Retrieval-Augmented Generation for Knowledge-I...
6,Author Names,"Mike Lewis†, Wen-tau Yih†, Tim Rocktäschel†‡, ...",application/pdf,['eng'],1,Retrieval-Augmented Generation for Knowledge-I...,Lists the authors of the research paper.,"Patrick Lewis,Ethan Perez,Aleksandra Piktus,Fa...","Facebook AI Research,University College London...",2020,Retrieval-Augmented Generation for Knowledge-I...,Retrieval-Augmented Generation for Knowledge-I...
9,NarrativeText,Large pre-trained language models have been sh...,application/pdf,['eng'],1,Retrieval-Augmented Generation for Knowledge-I...,This paragraph discusses the capabilities and ...,"Patrick Lewis,Ethan Perez,Aleksandra Piktus,Fa...","Facebook AI Research,University College London...",2020,Retrieval-Augmented Generation for Knowledge-I...,Abstract
11,NarrativeText,Pre-trained neural language models have been s...,application/pdf,['eng'],1,Retrieval-Augmented Generation for Knowledge-I...,This block discusses the advantages and disadv...,"Patrick Lewis,Ethan Perez,Aleksandra Piktus,Fa...","Facebook AI Research,University College London...",2020,Retrieval-Augmented Generation for Knowledge-I...,1 Introduction
...,...,...,...,...,...,...,...,...,...,...,...,...
216,FigureCaption,Table 7: Number of instances in the datasets u...,application/pdf,['eng'],19,Retrieval-Augmented Generation for Knowledge-I...,"Describes the content of Table 7, indicating t...","Patrick Lewis,Ethan Perez,Aleksandra Piktus,Fa...","Facebook AI Research,University College London...",2020,Retrieval-Augmented Generation for Knowledge-I...,G Parameters
217,Table,| Task | Train | Development | Test |\n|---|--...,application/pdf,['eng'],19,Retrieval-Augmented Generation for Knowledge-I...,Table 7: Number of instances in the datasets u...,"Patrick Lewis,Ethan Perez,Aleksandra Piktus,Fa...","Facebook AI Research,University College London...",2020,Retrieval-Augmented Generation for Knowledge-I...,G Parameters
218,NarrativeText,"parameters. The best performing ""closed-book"" ...",application/pdf,['eng'],19,Retrieval-Augmented Generation for Knowledge-I...,This block discusses the performance of differ...,"Patrick Lewis,Ethan Perez,Aleksandra Piktus,Fa...","Facebook AI Research,University College London...",2020,Retrieval-Augmented Generation for Knowledge-I...,G Parameters
220,NarrativeText,"In preliminary experiments, we observed that f...",application/pdf,['eng'],19,Retrieval-Augmented Generation for Knowledge-I...,This block discusses challenges encountered du...,"Patrick Lewis,Ethan Perez,Aleksandra Piktus,Fa...","Facebook AI Research,University College London...",2020,Retrieval-Augmented Generation for Knowledge-I...,H Retrieval Collapse


In [41]:
# Crear el ID utilizando el índice y el nombre del archivo
df['node_id'] = df.apply(lambda row: f"{row.name}_{row['filename']}", axis=1)

# Crear la columna 'previous_node_id' desplazando el ID hacia abajo
df['previous_node_id'] = df['node_id'].shift(1)

# Crear la columna 'next_node_id' desplazando el ID hacia arriba
df['next_node_id'] = df['node_id'].shift(-1)

df

,block_type,text,file_type,languages,page_number,filename,description,authors,research_lab,publication_year,title,section_title,node_id,previous_node_id,next_node_id
4,Author Names,"Patrick Lewis'?, Ethan Perez*,",application/pdf,['eng'],1,Retrieval-Augmented Generation for Knowledge-I...,"Lists the names of authors, potentially with a...","Patrick Lewis,Ethan Perez,Aleksandra Piktus,Fa...","Facebook AI Research,University College London...",2020,Retrieval-Augmented Generation for Knowledge-I...,Retrieval-Augmented Generation for Knowledge-I...,4_Retrieval-Augmented Generation for Knowledge...,None,5_Retrieval-Augmented Generation for Knowledge...
5,Author Names,"Aleksandra Piktus†, Fabio Petroni†, Vladimir K...",application/pdf,['eng'],1,Retrieval-Augmented Generation for Knowledge-I...,"Lists the names of authors, potentially affili...","Patrick Lewis,Ethan Perez,Aleksandra Piktus,Fa...","Facebook AI Research,University College London...",2020,Retrieval-Augmented Generation for Knowledge-I...,Retrieval-Augmented Generation for Knowledge-I...,5_Retrieval-Augmented Generation for Knowledge...,4_Retrieval-Augmented Generation for Knowledge...,6_Retrieval-Augmented Generation for Knowledge...
6,Author Names,"Mike Lewis†, Wen-tau Yih†, Tim Rocktäschel†‡, ...",application/pdf,['eng'],1,Retrieval-Augmented Generation for Knowledge-I...,Lists the authors of the research paper.,"Patrick Lewis,Ethan Perez,Aleksandra Piktus,Fa...","Facebook AI Research,University College London...",2020,Retrieval-Augmented Generation for Knowledge-I...,Retrieval-Augmented Generation for Knowledge-I...,6_Retrieval-Augmented Generation for Knowledge...,5_Retrieval-Augmented Generation for Knowledge...,9_Retrieval-Augmented Generation for Knowledge...
9,NarrativeText,Large pre-trained language models have been sh...,application/pdf,['eng'],1,Retrieval-Augmented Generation for Knowledge-I...,This paragraph discusses the capabilities and ...,"Patrick Lewis,Ethan Perez,Aleksandra Piktus,Fa...","Facebook AI Research,University College London...",2020,Retrieval-Augmented Generation for Knowledge-I...,Abstract,9_Retrieval-Augmented Generation for Knowledge...,6_Retrieval-Augmented Generation for Knowledge...,11_Retrieval-Augmented Generation for Knowledg...
11,NarrativeText,Pre-trained neural language models have been s...,application/pdf,['eng'],1,Retrieval-Augmented Generation for Knowledge-I...,This block discusses the advantages and disadv...,"Patrick Lewis,Ethan Perez,Aleksandra Piktus,Fa...","Facebook AI Research,University College London...",2020,Retrieval-Augmented Generation for Knowledge-I...,1 Introduction,11_Retrieval-Augmented Generation for Knowledg...,9_Retrieval-Augmented Generation for Knowledge...,12_Retrieval-Augmented Generation for Knowledg...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
216,FigureCaption,Table 7: Number of instances in the datasets u...,application/pdf,['eng'],19,Retrieval-Augmented Generation for Knowledge-I...,"Describes the content of Table 7, indicating t...","Patrick Lewis,Ethan Perez,Aleksandra Piktus,Fa...","Facebook AI Research,University College London...",2020,Retrieval-Augmented Generation for Knowledge-I...,G Parameters,216_Retrieval-Augmented Generation for Knowled...,214_Retrieval-Augmented Generation for Knowled...,217_Retrieval-Augmented Generation for Knowled...
217,Table,| Task | Train | Development | Test |\n|---|--...,application/pdf,['eng'],19,Retrieval-Augmented Generation for Knowledge-I...,Table 7: Number of instances in the datasets u...,"Patrick Lewis,Ethan Perez,Aleksandra Piktus,Fa...","Facebook AI Research,University College London...",2020,Retrieval-Augmented Generation for Knowledge-I...,G Parameters,217_Retrieval-Augmented Generation for Knowled...,216_Retrieval-Augmented Generation for Knowled...,218_Retrieval-Augmented Generation for Knowled...
218,NarrativeText,"parameters. The best performing ""closed-book"" ...",application/pdf,['eng'],19,Retrieval-Augmented Generatio

In [42]:
def create_node(row):
    # Crear el ID utilizando el índice y el nombre del archivo
    node_id = f"{row.name}_{row['filename']}"

    # Crear el nodo de texto con el ID, texto y metadatos
    node = TextNode(
        id=row['node_id'],
        text=row['text'],  # Usar la columna 'text' para el contenido del nodo
        metadata={
            'block_type': row['block_type'],
            'section_title': row['section_title'],
            'description': row['description'],
            'file_type': row['file_type'],
            'languages': row['languages'],
            'page_number': row['page_number'],
            'filename': row['filename'],
            'authors': row['authors'],
            'research_lab': row['research_lab'],
            'publication_year': row['publication_year'],
            'title': row['title']
        },
        excluded_llm_metadata_keys=['authors', 'research_lab'],
        excluded_embed_metadata_keys=['title']
    )
    if pd.notna(row['previous_node_id']):
        node.relationships[NodeRelationship.PREVIOUS] = RelatedNodeInfo(
            node_id=row['previous_node_id']
        )
    if pd.notna(row['next_node_id']):
        node.relationships[NodeRelationship.NEXT] = RelatedNodeInfo(
            node_id=row['next_node_id']
        )
    return node

In [43]:
df['node'] = df.apply(create_node, axis=1)

In [44]:
nodes = df['node'].tolist()
nodes[0]

TextNode(id_='c04e9272-4df6-4ef1-9349-f24acbdcdea2', embedding=None, metadata={'block_type': 'Author Names', 'section_title': 'Retrieval-Augmented Generation for Knowledge-Intensive NLP Tasks', 'description': 'Lists the names of authors, potentially with affiliations indicated by symbols.', 'file_type': 'application/pdf', 'languages': "['eng']", 'page_number': 1, 'filename': 'Retrieval-Augmented Generation for Knowledge-Intensive NLP Tasks.pdf', 'authors': 'Patrick Lewis,Ethan Perez,Aleksandra Piktus,Fabio Petroni,Vladimir Karpukhin,Naman Goyal,Heinrich KÃ¼ttler,Mike Lewis,Wen-tau Yih,Tim RocktÃ¤schel,Sebastian Riedel,Douwe Kiela', 'research_lab': 'Facebook AI Research,University College London,New York University', 'publication_year': 2020, 'title': 'Retrieval-Augmented Generation for Knowledge-Intensive NLP Tasks'}, excluded_embed_metadata_keys=['title'], excluded_llm_metadata_keys=['authors', 'research_lab'], relationships={<NodeRelationship.NEXT: '3'>: RelatedNodeInfo(node_id='5_Re

In [45]:
credentials_path = 'C:\\Agentes-RAG\\GCP-Credentials\\llms-433815-5e7ca2a0c045.json'
embedding = VertexIEmbeddings(credentials_path=credentials_path)
Settings.embed_model = embedding

model_name: text-embedding-004


In [46]:
embeded_nodes = embedding(nodes)
embeded_nodes[0]

TextNode(id_='c04e9272-4df6-4ef1-9349-f24acbdcdea2', embedding=[-0.040346380323171616, -0.022464128211140633, -0.05197039991617203, 0.0053776707500219345, -0.0019971011206507683, 0.0013902459759265184, 0.017468459904193878, 0.024134838953614235, 0.04877280816435814, -0.010017641820013523, -0.030863329768180847, -0.027494480833411217, 0.023229127749800682, 0.013211933895945549, 0.0018636994063854218, -0.021698765456676483, 0.006273228675127029, 0.053895555436611176, -0.0625259280204773, -0.002528643235564232, 0.0072318012826144695, -0.033708736300468445, 0.006138769444078207, -0.047183841466903687, -0.021933559328317642, -0.04295714572072029, 0.05227678641676903, -0.022620609030127525, -0.004851000849157572, -0.006953277625143528, 0.022087588906288147, 0.0642944946885109, 0.015330696478486061, 0.010571449995040894, -0.004879107233136892, 0.05753064900636673, -0.024754401296377182, 0.027319954708218575, 0.02849961631000042, -0.09810864180326462, -0.053667403757572174, 0.01394648384302854

In [47]:
# Guardar la lista de objetos a un archivo
with open('embeded_nodes.pkl', 'wb') as outp:  # 'wb' para escribir en modo binario
    pickle.dump(embeded_nodes, outp, pickle.HIGHEST_PROTOCOL)

In [18]:
# Leer la lista de objetos desde un archivo
with open('embeded_nodes.pkl', 'rb') as inp:  # 'rb' para leer en modo binario
    loaded_objects_list = pickle.load(inp)
loaded_objects_list[0]

TextNode(id_='8ac390ca-3ea4-4bae-b4a2-16b1de584638', embedding=[-0.040346380323171616, -0.022464128211140633, -0.05197039991617203, 0.0053776707500219345, -0.0019971011206507683, 0.0013902459759265184, 0.017468459904193878, 0.024134838953614235, 0.04877280816435814, -0.010017641820013523, -0.030863329768180847, -0.027494480833411217, 0.023229127749800682, 0.013211933895945549, 0.0018636994063854218, -0.021698765456676483, 0.006273228675127029, 0.053895555436611176, -0.0625259280204773, -0.002528643235564232, 0.0072318012826144695, -0.033708736300468445, 0.006138769444078207, -0.047183841466903687, -0.021933559328317642, -0.04295714572072029, 0.05227678641676903, -0.022620609030127525, -0.004851000849157572, -0.006953277625143528, 0.022087588906288147, 0.0642944946885109, 0.015330696478486061, 0.010571449995040894, -0.004879107233136892, 0.05753064900636673, -0.024754401296377182, 0.027319954708218575, 0.02849961631000042, -0.09810864180326462, -0.053667403757572174, 0.01394648384302854

In [48]:
index = VectorStoreIndex(nodes)

In [49]:
index.storage_context.persist(persist_dir='./data/index/vector_index_prueba')

In [50]:
index.docstore.get_all_document_hashes()

{'c7a67bf299e9f30b6345fb604cfe42e055d73e7b1d177c74851ca15ef16bd5aa': 'c04e9272-4df6-4ef1-9349-f24acbdcdea2',
 'e78fa67bf26b0d43dd8c990a0745feb5c4d98482dc66efcb48521dc16f044f94': '5f6d0882-e6fa-4ca6-9c0e-b27e5cc60257',
 'de423cb2d6bb54bb3927444af930185a1cb028eb8720355829cccf598b865199': '9a6ccc62-89a3-48a9-a5ba-f5becd646dae',
 'dedc44887d5b7120a2854a3bdfda9fe69f3675129770dd50c395d55de8a995f1': '36f747b9-1eed-423e-80ca-3a5b5c671e65',
 '046999e201589f407f019aca31e9a43c467c5465f6b2a450c85793e89ac587f7': '5f36ca8a-7fb4-416e-adba-90d260946cb8',
 '13a5c006c843644dcf84674f67678183a0ff0726a7d1516c9496be527e0e50a8': '48372a60-e0b7-47de-bc49-44ba335e9777',
 'c1da6b74efb96221f58945ef96d75f642cd200ac500e6fe360d5865dd058e3e5': '74f5a4fe-a96a-476b-8745-1bcd809ea25a',
 '3a98e6bf14a7c3f374e1a7c733b7e2beb7490db7ec71a2667f56dffa0d9e1ddf': '630b2e4e-43f6-49bc-a6c5-9cb6e804c4e7',
 '3feb9b9c7bd8f4aa46bc3a9a88a08e19e8dbccaef9be01f04d6e1c041151b64a': '5af91994-7b8b-4608-a960-6063b6bf6f3e',
 '548e59649c19112cb

In [51]:
index.docstore.get_all_ref_doc_info()

{}

In [54]:
index.docstore.get_document('d3fc8886-2d86-40e2-be54-f37422e67c1a')

TextNode(id_='d3fc8886-2d86-40e2-be54-f37422e67c1a', embedding=None, metadata={'block_type': 'NarrativeText', 'section_title': 'I Number of instances per dataset', 'description': 'This sentence describes the content of Table 7, indicating it shows the number of data points for training, development, and testing in each dataset used.', 'file_type': 'application/pdf', 'languages': "['eng']", 'page_number': 19, 'filename': 'Retrieval-Augmented Generation for Knowledge-Intensive NLP Tasks.pdf', 'authors': 'Patrick Lewis,Ethan Perez,Aleksandra Piktus,Fabio Petroni,Vladimir Karpukhin,Naman Goyal,Heinrich KÃ¼ttler,Mike Lewis,Wen-tau Yih,Tim RocktÃ¤schel,Sebastian Riedel,Douwe Kiela', 'research_lab': 'Facebook AI Research,University College London,New York University', 'publication_year': 2020, 'title': 'Retrieval-Augmented Generation for Knowledge-Intensive NLP Tasks'}, excluded_embed_metadata_keys=['title'], excluded_llm_metadata_keys=['authors', 'research_lab'], relationships={<NodeRelation

In [70]:
retriver = index.as_retriever(similarity_top_k=5)

In [73]:
r = retriver.retrieve('what is rag system?')
for x in map(lambda x: (x.text, x.score), r):
    print(x[0], x[1], sep='\n', end='\n\n')

To train the retriever and generator end-to-end, we treat the retrieved document as a latent variable. We propose two models that marginalize over the latent documents in different ways to produce a distribution over generated text. In one approach, RAG-Sequence, the model uses the same document to predict each target token. The second approach, RAG-Token, can predict each target token based on a different document. In the following, we formally introduce both models and then describe the pη and pθ components, as well as the training and decoding procedure.
0.5721262892645856

RAG-Token Model In the RAG-Token model we can draw a different latent document for each target token and marginalize accordingly. This allows the generator to choose content from several documents when producing an answer. Concretely, the top K documents are retrieved using the retriever, and then the generator produces a distribution for the next output token for each document, before marginalizing, and repeatin

In [76]:
from llama_index.llms.vertex import Vertex
from llama_index.core import VectorStoreIndex, get_response_synthesizer

llm = Vertex(model="gemini-1.5-pro-001")
Settings.llm = llm
llm.complete("Hello this is a sample text").text

'Please provide me with more context or information about what you would like me to do with the sample text "Hello this is a sample text". \n\nFor example, do you want me to:\n\n* Analyze the sentiment of the text?\n* Translate it into another language?\n* Identify the grammatical structure of the sentence?\n* Use it as a starting point for a story?\n\nOnce I understand your goal, I can be more helpful. 😊 \n'

In [88]:
query_engine = index.as_query_engine(verbose=True)

In [89]:
print(query_engine.query('what is rag system?').response)

This system uses retrieved documents to predict and generate text. It employs two different models, one that utilizes a single document for prediction and another that leverages multiple documents to produce an answer. 



In [80]:
print(query_engine.query('Que es un sistema rag?').response)

Es un sistema que combina un modelo de lenguaje pre-entrenado con una memoria no paramétrica, como un índice de documentos, para mejorar la generación de texto en tareas que requieren un conocimiento extenso. 



In [81]:
print(query_engine.query('Dame titulos de documentos que tengas?').response)

Retrieval-Augmented Generation for Knowledge-Intensive NLP Tasks. 



In [82]:
print(query_engine.query('Quienes son los autores de: Retrieval-Augmented Generation for Knowledge-Intensive NLP Tasks.').response)

Mike Lewis, Wen-tau Yih, Tim Rocktäschel, Sebastian Riedel, y Douwe Kiela. 



In [84]:
print(query_engine.query('Dame el bloque completo de informacion, donde diga que Mike Lewis, Wen-tau Yih, Tim Rocktäschel, Sebastian Riedel, y Douwe Kiela. son autores').response)

```json
{
  "block_type": "Author Names",
  "section_title": "Retrieval-Augmented Generation for Knowledge-Intensive NLP Tasks",
  "description": "Lists the authors of the research paper.",
  "file_type": "application/pdf",
  "languages": [
    "eng"
  ],
  "page_number": 1,
  "filename": "Retrieval-Augmented Generation for Knowledge-Intensive NLP Tasks.pdf",
  "publication_year": 2020,
  "title": "Retrieval-Augmented Generation for Knowledge-Intensive NLP Tasks",
  "authors": "Mike Lewis†, Wen-tau Yih†, Tim Rocktäschel†‡, Sebastian Riedel†‡, Douwe Kiela†"
}
```



In [85]:
print(query_engine.query('Puedes resumir el articulo: Retrieval-Augmented Generation for Knowledge-Intensive NLP Tasks.').response)

Este artículo explora modelos de lenguaje que combinan memoria paramétrica y no paramétrica para la generación de lenguaje. Se enfoca en modelos RAG que usan un modelo pre-entrenado seq2seq como memoria paramétrica y un índice vectorial denso de Wikipedia como memoria no paramétrica, a la que se accede con un recuperador neuronal pre-entrenado. 

El artículo compara dos formulaciones RAG: una que se condiciona en los mismos pasajes recuperados a través de toda la secuencia generada y otra que puede usar diferentes pasajes por token. Los modelos RAG se evalúan en una amplia gama de tareas de PNL de uso intensivo de conocimiento, incluyendo tareas de preguntas y respuestas de dominio abierto y tareas de generación de lenguaje. 

Los resultados muestran que los modelos RAG superan a los modelos seq2seq paramétricos y a las arquitecturas de recuperación y extracción específicas de la tarea en las tareas de preguntas y respuestas. Para las tareas de generación de lenguaje, los modelos RAG g

In [86]:
chat_engine = index.as_chat_engine(chat_mode="react", verbose=True)

In [90]:
response = chat_engine.chat("Hola soy Nicolas, me puedes dar informacion sobre el sistema RAG?")
print(response)

> Running step d9a03862-25db-4afc-9163-5c66d81063b4. Step input: Hola soy Nicolas, me puedes dar informacion sobre el sistema RAG?
Thought: The current language of the user is: Spanish. I need to use a tool to help me answer the question.
Action: query_engine_tool
Action Input: {'input': 'What is RAG?'}
Observation: RAG is a method that enhances pre-trained generation models by adding a non-parametric memory component. This approach uses a retriever to fetch relevant documents based on the input and a generator that utilizes these documents along with the input to produce the output. 

> Running step a5452aa1-d485-4292-8e34-23877a948d54. Step input: None
Thought: I can answer without using any more tools. I'll use the user's language to answer
Answer: Hola Nicolas, RAG son las siglas en inglés de "Recuperación Aumentada de Generación". Es un método que mejora los modelos de lenguaje pre-entrenados al proporcionarles acceso a información externa, como documentos o bases de datos. 

En l

In [91]:
chat_engine.chat('Recuerdas como me llamo?')

> Running step 2b29f316-a369-4978-8a62-e8c42f0272fd. Step input: Recuerdas como me llamo?
Thought: (Implicit) I can answer without any more tools!
Answer: Sí, recuerdo tu nombre. Te llamas Nicolas. 😊 



AgentChatResponse(response='Sí, recuerdo tu nombre. Te llamas Nicolas. 😊 \n', sources=[], source_nodes=[], is_dummy_stream=False, metadata=None)

In [95]:
chat_engine = index.as_chat_engine(chat_mode="context", verbose=True)
response = chat_engine.chat("Hola soy Nicolas, me puedes dar informacion sobre el sistema RAG?")
print(response)

¡Hola Nicolas! Con gusto te puedo dar información sobre el sistema RAG (Retrieval-Augmented Generation):

**¿Qué es RAG?**

RAG significa "Generación Aumentada por Recuperación". Es un nuevo enfoque en el Procesamiento del Lenguaje Natural (PNL) que combina la capacidad de recuperación de información de los motores de búsqueda con la generación de texto de los modelos de lenguaje. 

**¿Cómo funciona RAG?**

En lugar de generar texto basándose únicamente en el conocimiento aprendido durante el entrenamiento, RAG primero busca en una base de datos de documentos relevantes al tema dado. Luego, utiliza la información de estos documentos para generar una respuesta más completa y precisa.

**Ventajas de usar RAG:**

* **Respuestas más precisas y completas:** Al acceder a información externa, RAG puede generar respuestas que van más allá de su conocimiento pre-entrenado.
* **Mitigación de "alucinaciones":** Los modelos de lenguaje tradicionales a veces "inventan" información. RAG reduce este 

In [97]:
chat_engine = index.as_chat_engine(chat_mode="condense_plus_context", verbose=True)
response = chat_engine.chat("Hola soy Nicolas, me puedes dar informacion sobre el sistema RAG?")
print(response)

Condensed question: Hola soy Nicolas, me puedes dar informacion sobre el sistema RAG?
Context: block_type: NarrativeText
section_title: 2.1 Models
description: Explains how RAG can be applied to sequence classification tasks by treating the target class as a single-length target sequence, making RAG-Sequence and RAG-Token equivalent in this context.
file_type: application/pdf
languages: ['eng']
page_number: 3
filename: Retrieval-Augmented Generation for Knowledge-Intensive NLP Tasks.pdf
publication_year: 2020
title: Retrieval-Augmented Generation for Knowledge-Intensive NLP Tasks

Finally, we note that RAG can be used for sequence classiﬁcation tasks by considering the target class as a target sequence of length one, in which case RAG-Sequence and RAG-Token are equivalent.

block_type: NarrativeText
section_title: 2 Methods
description: Describes open-source code for RAG experiments and provides links to the code repository and an interactive demo.
file_type: application/pdf
languages:

In [98]:

response = chat_engine.chat("recuerdas como me llamo?")
print(response)

Condensed question: Standalone question: ¿Cómo te llamas? 

Context: block_type: Image
section_title: B Human Evaluation
description: Figure 4: Annotation interface for human evaluation of factuality. A pop-out for detailed instructions and a worked example appear when clicking "view tool guide".
file_type: application/pdf
languages: ['eng']
page_number: 17
filename: Retrieval-Augmented Generation for Knowledge-Intensive NLP Tasks.pdf
publication_year: 2020
title: Retrieval-Augmented Generation for Knowledge-Intensive NLP Tasks

This image shows a question from a survey or test. 

**Left side:**
* Instructions: "View full instructions" and "View tool guide"
* Note: Explains that some questions are control questions requiring accurate answers for responses to be accepted.
* Task:  "Indicate which one of the following sentences is more factually true with respect to the subject. Using the internet to check whether the sentences are true is encouraged."

**Right side:**
* Question: "Which

In [100]:
streaming_response = chat_engine.stream_chat("Hola soy Nicolas, me puedes dar informacion sobre el sistema RAG?")
for token in streaming_response.response_gen:
    print(token, end="")

Condensed question: Standalone question: Hola, ¿me puedes dar información sobre el sistema RAG? 

Context: block_type: NarrativeText
section_title: 4.1 Open-domain Question Answering
description: This paragraph discusses the performance of the RAG model on open-domain question answering tasks, comparing it to other state-of-the-art models. It highlights RAG's advantages, such as its flexibility and strong results without needing specialized pre-training or components like re-rankers or extractive readers.
file_type: application/pdf
languages: ['eng']
page_number: 5
filename: Retrieval-Augmented Generation for Knowledge-Intensive NLP Tasks.pdf
publication_year: 2020
title: Retrieval-Augmented Generation for Knowledge-Intensive NLP Tasks

Table 1 shows results for RAG along with state-of-the-art models. On all four open-domain QA tasks, RAG sets a new state of the art (only on the T5-comparable split for TQA). RAG combines the generation ﬂexibility of the “closed-book” (parametric only) 

In [109]:
chat_engine = index.as_chat_engine(chat_mode="react", verbose=False)
streaming_response = chat_engine.stream_chat("Hola como estas")
for token in streaming_response.response_gen:
    print(token, end="")

's language to answer
Answer: Hola, estoy bien, ¿cómo estás? 


In [108]:
chat_engine = index.as_chat_engine(chat_mode="react", verbose=False)
streaming_response = chat_engine.chat("Hola como estas")
print(streaming_response)

Hola, estoy bien, ¿y tú?
